# LangChain Study Plan Approval Sample Project

Setting up a minimal notebook using university study plan approval use-case.


In [29]:
# Install dependencies (run as code cell)
!python -m pip install --upgrade pip
!pip install langchain
# Optional, for structured validation
!pip install pydantic-ai
!pip install -U langchain-perplexity



In [45]:
import os
from typing import Literal
from langchain_perplexity import ChatPerplexity
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from pprint import pprint

import getpass

if "PPLX_API_KEY" not in os.environ:
    os.environ["PPLX_API_KEY"] = getpass.getpass("Enter your Perplexity API key: ")



## Minimal Agent Example

A structured evaluation output format study plan input and simulate a minimal agent's output.


In [31]:
class StudyPlanEvaluation(BaseModel):
    """
    Represents the evaluation of a student's study plan.
    """
    decision: Literal["GREEN", "YELLOW", "RED"] = Field(
        description="The final decision score for the plan."
    )
    reasoning: str = Field(
        description="A brief, point-by-point explanation for the decision, referencing the criteria."
    )
    alignment_ok: bool = Field(
        description="Whether the courses align with the student's specialization."
    )
    scheduling_ok: bool = Field(
        description="Whether there are any scheduling or exam conflicts."
    )


Knowledge base and input prompt 

In [51]:
KNOWLEDGE_BASE = """
# University Rules & Evaluation Criteria

## 1. Core Requirements
- A study plan must cover at least two semesters (one year).
- The evaluation must check for Scheduling conflicts, Alignment with specialization, and overall Workload.

## 2. Scoring System
- **GREEN**: Approved. The plan is perfect with no issues.
- **YELLOW**: Human Review Required. The plan has minor issues or potential risks that require a final decision from a human.
- **RED**: Rejected. The plan has critical faults in one or more criteria.

## 3. Course & Specialization Data
- **Specialization Blocks**:
  - Data Science: Requires courses 'CS501: Advanced Algorithms', 'CS502: Machine Learning', 'CS503: Big Data Analytics'.
  - Software Engineering: Requires courses 'SE501: Software Architecture', 'SE502: System Design', 'SE503: DevOps'.

## 4. Scheduling Constraints
- **Known Conflicts**:
  - 'CS501: Advanced Algorithms' lecture (Mon 10:00-12:00) conflicts with 'SE502: System Design' exercise (Mon 11:00-12:00).
- **Exam Spacing**:
  - A minimum of 3 full days is recommended between any two exam dates.
"""

STUDY_PLAN = """
# Student Study Plan Submission

- **Student Name**: Jane Doe
- **Specialization**: Data Science
- **Plan for Semesters**: Fall 2026, Spring 2027
- **Courses**:
  - 'CS501: Advanced Algorithms' (Exam: 2027-02-15)
  - 'CS502: Machine Learning' (Exam: 2027-02-20)
  - 'SE502: System Design' (Exam: 2027-03-10)
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert university administrator who approves student study plans. "
     "You must strictly follow the provided knowledge base to evaluate the plan. "
     "Your response must be structured according to the provided schema."
     "Reasoning must be concise and point-wise."),
    ("human",
     "Here is the knowledge base:\n--- KNOWLEDGE BASE ---\n{knowledge_base}\n\n"
     "Evaluate this study plan:\n--- STUDY PLAN ---\n{study_plan}")])

In [40]:
llm = ChatPerplexity(
    model="sonar",
    temperature=0.2
)
structured_llm = llm.with_structured_output(StudyPlanEvaluation)

# Create the Chain using Langchain Expression Language (LCEL)
# The '|' operator creates a seamless chain from prompt to model.
chain = prompt_template | structured_llm

In [48]:
# Run the Chain and Get the Structured Evaluation
def evaluate_plan_structured():
    """
    Runs the chain and prints the resulting Pydantic object.
    """
    
    # The chain's 'invoke' method runs the evaluation.
    response_object = chain.invoke({
        "knowledge_base": KNOWLEDGE_BASE,
        "study_plan": STUDY_PLAN
    })

    print("\n--- Study Plan Evaluation Result ---")
    print(f"Decision: {response_object.decision}")
    pprint(f"Reasoning: {response_object.reasoning}")
    print(f"Alignment OK: {response_object.alignment_ok}")
    print(f"Scheduling OK: {response_object.scheduling_ok}")
    print(f"\nType of response: {type(response_object)}")

In [53]:
for _ in range(5):
    print("\n==============================\n")
    evaluate_plan_structured()




--- Study Plan Evaluation Result ---
Decision: YELLOW
('Reasoning: 1. The study plan covers two semesters (Fall 2026 and Spring '
 '2027), satisfying the minimum duration requirement. 2. The student’s '
 'specialization is Data Science, which requires courses CS501, CS502, and '
 'CS503. The plan includes CS501 and CS502 but lacks CS503, so the '
 'specialization alignment is incomplete. 3. There is a known scheduling '
 'conflict between CS501 lecture and SE502 exercise on Mondays 10:00-12:00 and '
 '11:00-12:00 respectively; since both courses are in the plan, this conflict '
 'exists. 4. Exam spacing is adequate with at least 3 full days between exams '
 '(Feb 15, Feb 20, Mar 10). 5. Overall workload and semester coverage are '
 'acceptable. Due to the missing required course for the specialization and '
 'the scheduling conflict, the plan has minor but critical issues requiring '
 'human review.')
Alignment OK: False
Scheduling OK: False

Type of response: <class '__main__.Study